# Auto Encoders

Notebook ini berdasarkan kursus __Deep Learning A-Z™: Hands-On Artificial Neural Networks__ di Udemy. [Lihat Kursus](https://www.udemy.com/deeplearning/).

## Informasi Notebook
- __notebook name__: `taruma_udemy_autoencoders`
- __notebook version/date__: `1.0.0`/`20190801`
- __notebook server__: Google Colab
- __python version__: `3.6`
- __pytorch version__: `1.1.0`


In [0]:
#### NOTEBOOK DESCRIPTION

from datetime import datetime

NOTEBOOK_TITLE = 'taruma_udemy_autoencoders'
NOTEBOOK_VERSION = '1.0.0'
NOTEBOOK_DATE = 1 # Set 1, if you want add date classifier

NOTEBOOK_NAME = "{}_{}".format(
    NOTEBOOK_TITLE, 
    NOTEBOOK_VERSION.replace('.','_')
)
PROJECT_NAME = "{}_{}{}".format(
    NOTEBOOK_TITLE, 
    NOTEBOOK_VERSION.replace('.','_'), 
    "_" + datetime.utcnow().strftime("%Y%m%d_%H%M") if NOTEBOOK_DATE else ""
)

print(f"Nama Notebook: {NOTEBOOK_NAME}")
print(f"Nama Proyek: {PROJECT_NAME}")

Nama Notebook: taruma_udemy_autoencoders_1_0_0
Nama Proyek: taruma_udemy_autoencoders_1_0_0_20190801_0925


In [0]:
#### System Version
import sys, torch
print("versi python: {}".format(sys.version))
print("versi pytorch: {}".format(torch.__version__))

versi python: 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]
versi pytorch: 1.1.0


In [0]:
#### Load Notebook Extensions
%load_ext google.colab.data_table

In [0]:
#### Download dataset
# ref: https://grouplens.org/datasets/movielens/
!wget -O autoencoders.zip "https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P16-AutoEncoders.zip"
!unzip autoencoders.zip

--2019-08-01 09:25:40--  https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P16-AutoEncoders.zip
Resolving sds-platform-private.s3-us-east-2.amazonaws.com (sds-platform-private.s3-us-east-2.amazonaws.com)... 52.219.80.168
Connecting to sds-platform-private.s3-us-east-2.amazonaws.com (sds-platform-private.s3-us-east-2.amazonaws.com)|52.219.80.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17069342 (16M) [application/zip]
Saving to: ‘autoencoders.zip’

autoencoders.zip    100%[===================>]  16.28M  34.2MB/s    in 0.5s    

2019-08-01 09:25:40 (34.2 MB/s) - ‘autoencoders.zip’ saved [17069342/17069342]

Archive:  autoencoders.zip
   creating: AutoEncoders/
  inflating: AutoEncoders/ae.py      
   creating: __MACOSX/
   creating: __MACOSX/AutoEncoders/
  inflating: __MACOSX/AutoEncoders/._ae.py  
  inflating: AutoEncoders/ml-100k.zip  
  inflating: AutoEncoders/ml-1m.zip  


In [0]:
# Karena ada file .zip dalam direktori, harus diekstrak lagi.
# ref: https://askubuntu.com/q/399951
# ref: https://unix.stackexchange.com/q/12902
!find AutoEncoders -type f -name '*.zip' -exec unzip -d AutoEncoders {} \;

Archive:  AutoEncoders/ml-100k.zip
   creating: AutoEncoders/ml-100k/
  inflating: AutoEncoders/ml-100k/allbut.pl  
   creating: AutoEncoders/__MACOSX/
   creating: AutoEncoders/__MACOSX/ml-100k/
  inflating: AutoEncoders/__MACOSX/ml-100k/._allbut.pl  
  inflating: AutoEncoders/ml-100k/mku.sh  
  inflating: AutoEncoders/__MACOSX/ml-100k/._mku.sh  
  inflating: AutoEncoders/ml-100k/README  
  inflating: AutoEncoders/__MACOSX/ml-100k/._README  
  inflating: AutoEncoders/ml-100k/u.data  
  inflating: AutoEncoders/__MACOSX/ml-100k/._u.data  
  inflating: AutoEncoders/ml-100k/u.genre  
  inflating: AutoEncoders/__MACOSX/ml-100k/._u.genre  
  inflating: AutoEncoders/ml-100k/u.info  
  inflating: AutoEncoders/__MACOSX/ml-100k/._u.info  
  inflating: AutoEncoders/ml-100k/u.item  
  inflating: AutoEncoders/__MACOSX/ml-100k/._u.item  
  inflating: AutoEncoders/ml-100k/u.occupation  
  inflating: AutoEncoders/__MACOSX/ml-100k/._u.occupation  
  inflating: AutoEncoders/ml-100k/u.user  
  inflating

In [0]:
#### Atur dataset path
DATASET_DIRECTORY = 'AutoEncoders/'

In [0]:
def showdata(dataframe):
    print('Dataframe Size: {}'.format(dataframe.shape))
    return dataframe

# STEP 1-5 DATA PREPROCESSING

In [0]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
movies = pd.read_csv(DATASET_DIRECTORY + 'ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
showdata(movies).head(10)

Dataframe Size: (3883, 3)


,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [0]:
users = pd.read_csv(DATASET_DIRECTORY + 'ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
showdata(users).head(10)

Dataframe Size: (6040, 5)


,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [0]:
ratings = pd.read_csv(DATASET_DIRECTORY + 'ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
showdata(ratings).head(10)

Dataframe Size: (1000209, 4)


,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [0]:
# Preparing the training set and the test set
training_set = pd.read_csv(DATASET_DIRECTORY + 'ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv(DATASET_DIRECTORY + 'ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

In [0]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [0]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [0]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [0]:
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

# STEP 6-7

In [0]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# STEP 8-10

In [0]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 1.7663983791313438
epoch: 2 loss: 1.0965944818481448
epoch: 3 loss: 1.0533398732955221
epoch: 4 loss: 1.0383018413922185
epoch: 5 loss: 1.0308177439541621
epoch: 6 loss: 1.026551124053685
epoch: 7 loss: 1.023840092408676
epoch: 8 loss: 1.021978586980373
epoch: 9 loss: 1.0206570638587025
epoch: 10 loss: 1.0196462708959995
epoch: 11 loss: 1.0187753163243505
epoch: 12 loss: 1.018512555740381
epoch: 13 loss: 1.0178744683018195
epoch: 14 loss: 1.0174755647701952
epoch: 15 loss: 1.0170719470478082
epoch: 16 loss: 1.017201642832892
epoch: 17 loss: 1.0163239136444078
epoch: 18 loss: 1.0165747767066637
epoch: 19 loss: 1.0162508415906395
epoch: 20 loss: 1.0162299744574526
epoch: 21 loss: 1.0160825599663328
epoch: 22 loss: 1.0159708620648906
epoch: 23 loss: 1.0159037432204494
epoch: 24 loss: 1.0156694047908619
epoch: 25 loss: 1.0156815102111703
epoch: 26 loss: 1.0154590358153581
epoch: 27 loss: 1.0152956203593735
epoch: 28 loss: 1.0151429122142581
epoch: 29 loss: 1.0127277229574954

In [0]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.9503542203018388
